In [1]:
import sys

sys.path.append("..")

In [2]:
import os

sys.path.insert(0, os.path.expanduser("~/projects/bpack"))

In [3]:
from bpack.tools.codegen import FlatDescriptorCodeGenerator

In [4]:
from s1isp.descriptors import SecondaryHeader

code_generator = FlatDescriptorCodeGenerator(SecondaryHeader)
code_generator.imports["s1isp.descriptors"].discard("REF_FREQ")
code_generator.imports["s1isp.constants"].update(["REF_FREQ", "SYNC_MARKER"])
code_generator.imports["s1isp.descriptors"].update([
    "DatationService",
    "FixedAncillaryDataService",
    "SubCommutatedAncillaryDataService",
    "CountersService",
    "SasData",
    "SesData",
    "RadarConfigurationSupportService",
    "RadarSampleCountService",
])
code_generator.module_docstring = (
    "Flat version of the Sentinel-1 ISP SecondaryHeader."
)
code_generator.post_code = '''
    # === Services ===
    def get_datation(self) -> DatationService:
        """Return a DatationService instance."""
        return DatationService(self.coarse_time, self.fine_time)

    def get_fixed_ancillary_data(self) -> FixedAncillaryDataService:
        """Return a FixedAncillaryDataService instance."""
        return FixedAncillaryDataService(
            self.sync_marker,
            self.data_take_id,
            self.ecc_num,
            self.test_mode,
            self.rx_channel_id,
            self.instrument_configuration_id,
        )

    def get_subcom_ancillary_data(self) -> SubCommutatedAncillaryDataService:
        """Return a SubCommutatedAncillaryDataService instance."""
        return SubCommutatedAncillaryDataService(
            self.data_word_index,
            self.data_word,
        )

    def get_counters(self) -> CountersService:
        """Return a CountersService instance."""
        return CountersService(self.space_packet_count, self.pri_count)

    def get_sas(self) -> SasData:
        """Return a SasData instance."""
        return SasData(
            self.ssb_flag,
            self.polarization,
            self.temperature_compensation,
            self._dynamic_data,
            self._beam_address,
        )

    def get_ses(self) -> SesData:
        """Return a SesData instance."""
        return SesData(
            self.cal_mode,
            self.tx_pulse_number,
            self.signal_type,
            self.swap,
            self.swath_number,
        )

    def get_radar_configuration_support(
        self
    ) -> RadarConfigurationSupportService:
        """Return a RadarConfigurationSupportService instance."""
        return RadarConfigurationSupportService(
            self.error_flag,
            self.baq_mode,
            self.baq_block_length,
            self.range_decimation,
            self.rx_gain,
            self.tx_ramp_rate,
            self.tx_pulse_start_freq,
            self.tx_pulse_length,
            self.rank,
            self.pri,
            self.swst,
            self.swl,
            self.get_sas(),
            self.get_ses(),
        )

    def get_radar_sample_count(self) -> RadarSampleCountService:
        """Return a RadarSampleCountService instance."""
        return RadarSampleCountService(self.number_of_quads)
'''

In [5]:
code = code_generator.get_code(imports=True)

In [6]:
from s1isp.constants import SYNC_MARKER

code = code.replace(str(SYNC_MARKER), "SYNC_MARKER")

In [7]:
if True:
    with open("flat_descriptors.py", "w") as fd:
        fd.write(code)

In [8]:
from flat_descriptors import SecondaryHeader as FlatSecondaryHeader

In [9]:
with open("../tests/data/000408-echo.dat", "rb") as fd:
    data = fd.read()

from s1isp.constants import (
    PRIMARY_HEADER_SIZE as PHSIZE,
    SECONDARY_HEADER_SIZE as SHSIZE,
)

data = data[PHSIZE : PHSIZE + SHSIZE]

In [10]:
sh = SecondaryHeader.frombytes(data)

In [11]:
fsh = FlatSecondaryHeader.frombytes(data)

In [12]:
sh

SecondaryHeader(datation=DatationService(coarse_time=1276273467, fine_time=61863), fixed_ancillary_data=FixedAncillaryDataService(sync_marker=892270675, data_take_id=87747936, ecc_num=<EEccNumber.s3_no_ical: 13>, test_mode=<ETestMode.default: 0>, rx_channel_id=<ERxChannelId.rxv: 0>, instrument_configuration_id=1), subcom_ancillary_data=SubCommutatedAncillaryDataService(data_word_index=25, data_word=b'\xbe\xa3'), counters=CountersService(space_packet_count=408, pri_count=4427), radar_configuration_support=RadarConfigurationSupportService(error_flag=False, baq_mode=<EBaqMode.FDBAQ_MODE_0: 12>, baq_block_length=31, range_decimation=<ERangeDecimation.x4_on_9: 4>, rx_gain=12, tx_ramp_rate=34770, tx_pulse_start_freq=12970, tx_pulse_length=1658, rank=10, pri=19499, swst=5271, swl=12178, sas=SasData(ssb_flag=False, polarization=<EPolarization.v_vh: 7>, temperature_compensation=<ETemperatureCompensation.fe_on_ta_on: 3>, _dynamic_data=2, _beam_address=0), ses=SesData(cal_mode=<ECalMode.pcc2_ical

In [13]:
fsh

SecondaryHeader(coarse_time=1276273467, fine_time=61863, sync_marker=892270675, data_take_id=87747936, ecc_num=<EEccNumber.s3_no_ical: 13>, test_mode=<ETestMode.default: 0>, rx_channel_id=<ERxChannelId.rxv: 0>, instrument_configuration_id=1, data_word_index=25, data_word=b'\xbe\xa3', space_packet_count=408, pri_count=4427, error_flag=False, baq_mode=<EBaqMode.FDBAQ_MODE_0: 12>, baq_block_length=31, range_decimation=<ERangeDecimation.x4_on_9: 4>, rx_gain=12, tx_ramp_rate=34770, tx_pulse_start_freq=12970, tx_pulse_length=1658, rank=10, pri=19499, swst=5271, swl=12178, ssb_flag=False, polarization=<EPolarization.v_vh: 7>, temperature_compensation=<ETemperatureCompensation.fe_on_ta_on: 3>, _dynamic_data=2, _beam_address=0, cal_mode=<ECalMode.pcc2_ical_interleaved: 0>, tx_pulse_number=2, signal_type=<ESignalType.echo: 0>, swap=False, swath_number=2, number_of_quads=10779)

In [14]:
assert sh.datation.coarse_time == fsh.coarse_time
assert sh.datation.fine_time == fsh.fine_time

In [15]:
assert sh.fixed_ancillary_data.sync_marker == fsh.sync_marker
assert sh.fixed_ancillary_data.data_take_id == fsh.data_take_id
assert sh.fixed_ancillary_data.ecc_num == fsh.ecc_num
assert sh.fixed_ancillary_data.test_mode == fsh.test_mode
assert sh.fixed_ancillary_data.rx_channel_id == fsh.rx_channel_id
assert (
    sh.fixed_ancillary_data.instrument_configuration_id
    == fsh.instrument_configuration_id
)

In [16]:
assert sh.subcom_ancillary_data.data_word_index == fsh.data_word_index
assert sh.subcom_ancillary_data.data_word == fsh.data_word

In [17]:
assert sh.counters.space_packet_count == fsh.space_packet_count
assert sh.counters.pri_count == fsh.pri_count

In [18]:
assert sh.radar_configuration_support.error_flag == fsh.error_flag
assert sh.radar_configuration_support.baq_mode == fsh.baq_mode
assert sh.radar_configuration_support.baq_block_length == fsh.baq_block_length
assert sh.radar_configuration_support.range_decimation == fsh.range_decimation
assert sh.radar_configuration_support.rx_gain == fsh.rx_gain
assert sh.radar_configuration_support.tx_ramp_rate == fsh.tx_ramp_rate
assert (
    sh.radar_configuration_support.tx_pulse_start_freq
    == fsh.tx_pulse_start_freq
)
assert sh.radar_configuration_support.tx_pulse_length == fsh.tx_pulse_length
assert sh.radar_configuration_support.rank == fsh.rank
assert sh.radar_configuration_support.pri == fsh.pri
assert sh.radar_configuration_support.swst == fsh.swst
assert sh.radar_configuration_support.swl == fsh.swl

In [19]:
assert sh.radar_configuration_support.sas.ssb_flag == fsh.ssb_flag
assert sh.radar_configuration_support.sas.polarization == fsh.polarization
assert (
    sh.radar_configuration_support.sas.temperature_compensation
    == fsh.temperature_compensation
)
assert sh.radar_configuration_support.sas._dynamic_data == fsh._dynamic_data
assert sh.radar_configuration_support.sas._beam_address == fsh._beam_address

In [20]:
assert sh.radar_configuration_support.ses.cal_mode == fsh.cal_mode
assert (
    sh.radar_configuration_support.ses.tx_pulse_number == fsh.tx_pulse_number
)
assert sh.radar_configuration_support.ses.signal_type == fsh.signal_type
assert sh.radar_configuration_support.ses.swap == fsh.swap
assert sh.radar_configuration_support.ses.swath_number == fsh.swath_number

In [21]:
assert sh.radar_sample_count.number_of_quads == fsh.number_of_quads

In [22]:
assert sh.radar_configuration_support.sas == fsh.get_sas()
assert sh.radar_configuration_support.ses == fsh.get_ses()

assert sh.datation == fsh.get_datation()
assert sh.fixed_ancillary_data == fsh.get_fixed_ancillary_data()
assert sh.subcom_ancillary_data == fsh.get_subcom_ancillary_data()
assert sh.counters == fsh.get_counters()
assert sh.radar_configuration_support == fsh.get_radar_configuration_support()
assert sh.radar_sample_count == fsh.get_radar_sample_count()